In [1]:
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from selenium.common.exceptions import WebDriverException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup as bs
from time import sleep
import requests
import pandas as pd
import numpy as np

In [2]:
# Make a url list
url_base = 'https://hoopshype.com/salaries/players/'
url_list = []
a = 2021

while a >= 1996:
    b = a+1
    new_url = url_base + str(a) + "-" + str(b)
    a -=1
    url_list.append(new_url)

In [3]:
url_list[:5]

['https://hoopshype.com/salaries/players/2021-2022',
 'https://hoopshype.com/salaries/players/2020-2021',
 'https://hoopshype.com/salaries/players/2019-2020',
 'https://hoopshype.com/salaries/players/2018-2019',
 'https://hoopshype.com/salaries/players/2017-2018']

In [ ]:
dataframe_list = []
n=1 # Make a counter to see how many webpages are scraped

# Scrape stat table for each url in url_list
for url in url_list:

    # provide path to the Edge driver
    s=Service("C:\\Users\sandr\OneDrive\Desktop\Drivers\msedgedriver.exe")
    driver = webdriver.Edge(service=s)
    
    # Make python pause for some time
    sleep(2)

    # Get the page HTML
    try:
        driver.get(url)
        # Make python pause for some time
        sleep(3)
    except WebDriverException:
        print("Webdriver is acting immediately after ‘closing’ the browser")

    # Make python pause for some time
    sleep(2)

    # Use BeautifulSoup and lxml extension to parse, save in variable nba_soup
    nba_soup = bs(driver.page_source, 'lxml')
    
    # Locate the table on the page
    table = nba_soup.find('table', class_='hh-salaries-ranking-table hh-salaries-table-sortable responsive')
   
    # Find all the headers in the columns
    headers=[]
    for i in table.find_all('td')[1:4]:
        title = i.text.replace("\n", "").strip()
        headers.append(title)
    
    # Create a dataframe using the headers as column names
    p_salary = pd.DataFrame(columns = headers) 
    
    # Create a for loop to fill p_salary
    for i in table.find_all('tr')[1:]:
        row_data = i.find_all('td')[1:]
        row =[j.text.replace("\n", "").strip() for j in row_data] 
        length = len(p_salary)
        p_salary.loc[length] = row

    print("Completed:", n, "webpage(s) scraped.")
    n +=1 # add 1 to counter every successful page retrieved
        
    # Close all browsers
    driver.quit()

    # Append table to list of dataframes
    dataframe_list.append(p_salary)


Completed: 1 webpage(s) scraped.
Completed: 2 webpage(s) scraped.
Completed: 3 webpage(s) scraped.
Completed: 4 webpage(s) scraped.
Completed: 5 webpage(s) scraped.


In [ ]:
dataframe_list

In [ ]:
test = dataframe_list[0].append(dataframe_list[1], ignore_index=True)
test[650:700]

In [ ]:
# Concat all the dataframes into one table
salaries = pd.concat(dataframe_list, ignore_index=True)
salaries

In [ ]:
player1 = salaries[:][salaries['Player'].notna()]
player1

In [ ]:
test = dataframe_list[0][dataframe_list[0]['Player'] == 'Stephen Curry']
test

In [ ]:
test2 = dataframe_list[1][dataframe_list[1]['Player'] == 'Stephen Curry']
test2

In [ ]:
test3 = pd.merge(test, test2, on='Player', how='right')

In [ ]:
test3

In [ ]:
curry = pd.read_csv('curry.csv', encoding='utf-8', index_col=[0])

In [ ]:
curry

In [ ]:
curry_sal = pd.merge(curry, test3, left_on="PLAYER", right_on="Player")
curry_sal

In [ ]:
curry_sal2 = pd.merge(curry, test3.set_index("Player"), left_on="PLAYER", right_index=True)
curry_sal2

In [ ]:
# Concat all the dataframes into one table
p_salaries = pd.concat(dataframe_list, ignore_index=True)
p_salaries